# RSA Encryption Scheme

## Group-11 
+ Ritu Thombre (BT17CSE084)

# <a id='index'>Index</a>

+ <a href='#key'>1. Generation of Keys</a>
+ <a href='#encrypt'>2. Encrypting plain_text</a>
+ <a href='#decrypt'>3. Decrypting cipher_text</a>
+ <a href='#aux'>4. Auxiliary functions</a>
+ <a href='#implementation'>5. Implementation</a>
    + <a href='#6.1'>5.1 Generating Keys</a>
    + <a href='#6.2'>5.2 Generating plain_text</a>
    + <a href='#6.3'>5.3 Encrypting plain_text to generate cipher_text</a>
    + <a href='#6.4'>5.4 Decrypting cipher_text to generate plain_text</a>
    + <a href='#6.5'>5.5 Checking if decrypted plain_text is same as original plain_text</a>
+ <a href='#final'>6. Final Remarks</a>

In [21]:
import numpy as np
import math
import gmpy2
from gmpy2 import powmod,mpz,isqrt,invert

# <a id='key'>1. Generation of Keys</a>
<a href='#index'>Go back to the top</a>

***Key Generation In RSA :***

+ Choose 2 primes p,q using gmpy
+ n = p*q and $ \phi (n) = (p-1)*(q-1) $
+ Choose $ e $ in $ [2,\phi(n)-2] $ such that $ e $ and $ \phi(n) $ are coprimes i.e their gcd is 1
+ Choose $ d = e^{-1} mod(\phi(n)) $
+ Return ***Public keys :*** $ (n,e) $
+ Return ***Private key :*** $ (d) $

In [22]:
def generate_keys():
    # prime number of 25 digits i.e 84 bits
    temp = 1000000000000000000000000
    random_add1 = np.random.randint(1,1000000)
    random_add2 = np.random.randint(1000000,2000000)
    p = int(gmpy2.next_prime(temp + random_add1))
    q = int(gmpy2.next_prime(temp + random_add2))
    n = p*q
    phi = (p-1)*(q-1)
    e = 2
    while True:
        if gmpy2.gcd(phi,e) != 1:
            e = e + 1
        else :
            break
    d = gmpy2.invert(e,phi)
    return n,e,d

# <a id='encrypt'>2. Encrypting plain_text blocks</a>
<a href='#index'>Go back to the top</a>

***Encyption in RSA :***

***Input :*** $ Plain Text = [p_1,p_2,p_3,..........,p_k] $

***Output :*** $ Cipher Text = [c_1,c_2,c_3,..........,c_k] $

where

$ c_{i} = p_{i}^e mod (n) $


In [23]:
def encrypt(plain_text_blocks,public_keys):
    cipher_text_blocks = []
    n,e = public_keys
    for plain_text in plain_text_blocks:
        cipher_text = (gmpy2.powmod(plain_text,e,n))
        cipher_text_blocks.append(cipher_text)
    return cipher_text_blocks

# <a id='decrypt'>3. Decrypting cipher_text</a>
<a href='#index'>Go back to the top</a>

***Decryption in RSA :***

***Input :*** $ Cipher Text = [c_1,c_2,c_3,..........,c_k] $

***Output :*** $ Plain Text = [p_1,p_2,p_3,..........,p_k] $

$ p_{i} = c_{i}^d mod (n) $

In [24]:
def decrypt(cipher_text_blocks,secret_key,public_keys):
    n,e = public_keys
    d = secret_key
    decypted_plain_text_blocks = []
    for cipher_text in cipher_text_blocks:
        plain_text = (gmpy2.powmod(cipher_text,d,n))
        decypted_plain_text_blocks.append(plain_text)
    return decypted_plain_text_blocks

# <a id='aux'>4. Auxiliary functions</a>
<a href='#index'>Go back to the top</a>

+ get_blocks() function converts the plain_text into blocks of plain_text by using the ***blocksize = 12***.
    + In this case, since prime used is 84 bits long i.e 25 digits long, it can encrypt upto 25 digit number
    + Since, each aplha-numeric character gives 2 digit plain_text, so we can use maximum of ***12 alphabets in a block***
+ format_plain_text() takes in the alphabetical plain_text blocks and returns their corresponding numeric plain_text blocks
+ def format_decrypted_plain_text() takes in the numeric_decrypted_plain_text blocks and return a single original plain text.

In [25]:
# taken in 'Hello World!!!' returns ['Hello World!','!!']
def get_blocks(PT,block_size=12):
    blocks = []
    i = 0
    while i<len(PT):
        temp_str=''
        if i+block_size-1 < len(PT):
            temp_str=temp_str+PT[i:i+block_size]
        else :
            temp_str=temp_str+PT[i::]
        blocks.append(temp_str)
        i=i+block_size
    return blocks
        
# covert plain_text block from characters to the numbers
def format_plain_text(PT):
    plain_text_blocks = []
    for block in PT:
        plain_text = 0
        for i in range(len(block)):
            # for 'd'
            if ord(block[i]) == 100:
                plain_text = plain_text*100 + 28
            # between (101,127)
            elif ord(block[i])>100:
                plain_text = plain_text*100 + (ord(block[i])-100)
            else :
                plain_text = plain_text*100 + (ord(block[i]))
        plain_text_blocks.append(plain_text)
    return plain_text_blocks

# convert numeric decypted_plain_text_blocks into a single plain text of characters
def format_decrypted_plain_text(decypted_plain_text_blocks):
    plain_text_blocks = []
    for dc_pt in decypted_plain_text_blocks:
        plain_text = ''
        temp = dc_pt
        # for 'd' temp = 28
        while temp > 0:
            if temp%100 == 28:
                plain_text = plain_text + 'd'
            elif (temp%100) in range(0,27):
                plain_text = plain_text + chr((temp%100)+100)
            else :
                plain_text = plain_text + chr((temp%100))
            temp = temp//100
        plain_text = plain_text[::-1] 
        plain_text_blocks.append(plain_text)
    final_plain_text = ''
    for plain_text_block in plain_text_blocks:
        final_plain_text = final_plain_text + plain_text_block
    return final_plain_text

# <a id='implementation'>5. Implementation</a>
<a href='#index'>Go back to the top</a>

### <a id='6.1'>5.1 Generating Keys</a>

In [26]:
n,e,d = generate_keys()

In [27]:
public_keys = (n,e)
secret_key = d
print("\nPublic Key :")
print('n :',n)
print('e :',e)
print("Secret Key :\nd :",d)


Public Key :
n : 1000000000000000001341354000000000000281177501013
e : 7
Secret Key :
d : 285714285714285714668957714285714285794621759903


### <a id='6.2'>5.2 Generating plain_text</a>

In [28]:
PT = input("\nEnter Plain Text to encrypt : ")

original_plain_text = PT


Enter Plain Text to encrypt : Schrodinger's cat is dead and alive at the same time. This cat represents the (|psi_cat>=0.5*|psi_cat_alive> + 0.5*|psi_cat_dead>) i.e superposition of wave-functions representing cat being dead and alive at the same time.


In [29]:
block_size = 12
PT = get_blocks(PT,block_size)
print('\nPlain Text after converting to blocks',PT)


Plain Text after converting to blocks ["Schrodinger'", 's cat is dea', 'd and alive ', 'at the same ', 'time. This c', 'at represent', 's the (|psi_', 'cat>=0.5*|ps', 'i_cat_alive>', ' + 0.5*|psi_', 'cat_dead>) i', '.e superposi', 'tion of wave', '-functions r', 'epresenting ', 'cat being de', 'ad and alive', ' at the same', ' time.']


In [30]:
plain_text_blocks = format_plain_text(PT)
print('\nPlain text blocks after formatting to numbers:',plain_text_blocks)


Plain text blocks after formatting to numbers: [839904141128051003011439, 153299971632051532280197, 283297102832970805180132, 971632160401321597090132, 160509014632840405153299, 971632140112140115011016, 153216040132402412150595, 999716626148465342241215, 59599971695970805180162, 324332484653422412150595, 999716952801972862413205, 460132151712011412111505, 160511103211023219971801, 450217109916051110153214, 11214011501101605100332, 999716329801051003322801, 972832971028329708051801, 329716321604013215970901, 321605090146]


### <a id='6.3'>5.3 Encrypting plain_text to generate cipher_text</a>

In [31]:
cipher_text_blocks = encrypt(plain_text_blocks,public_keys)
print("\nCipher Text Blocks After RSA encryption :",cipher_text_blocks)


Cipher Text Blocks After RSA encryption : [mpz(774508198888187580587946737302815984351963186940), mpz(423279653715063599349011885640360111585672014163), mpz(669318001393529679708414629654112663820426265801), mpz(484340170643646038968628612182983150988761717386), mpz(125357411837221794225861460606317557411951790293), mpz(900791586397964162054100435523991156600042369828), mpz(842034793216642839840652761265394917139789361424), mpz(525451966087518281105034757125993599524720967904), mpz(491844854924193609627871362517676665473667147766), mpz(268342633309175463394382444910115704506252208646), mpz(17328679049235209240729641761810232631615686518), mpz(636825521776325957537174217280432436054741168920), mpz(287662996948861326539028277866028912382971579859), mpz(84358182807846399870970612886459670298476405514), mpz(352664057898467977239632367271176278443088183175), mpz(18101567976458017218131068155141512293448264382), mpz(81524323116989350084674286092231000859897440799), mpz(665621393475609092907

### <a id='6.4'>5.4 Decrypting cipher_text to generate plain_text</a>

In [32]:
decypted_plain_text_blocks = decrypt(cipher_text_blocks,secret_key,public_keys)
print("\nPlain Text blocks after decryption of Cipher Text blocks :",decypted_plain_text_blocks)


Plain Text blocks after decryption of Cipher Text blocks : [mpz(839904141128051003011439), mpz(153299971632051532280197), mpz(283297102832970805180132), mpz(971632160401321597090132), mpz(160509014632840405153299), mpz(971632140112140115011016), mpz(153216040132402412150595), mpz(999716626148465342241215), mpz(59599971695970805180162), mpz(324332484653422412150595), mpz(999716952801972862413205), mpz(460132151712011412111505), mpz(160511103211023219971801), mpz(450217109916051110153214), mpz(11214011501101605100332), mpz(999716329801051003322801), mpz(972832971028329708051801), mpz(329716321604013215970901), mpz(321605090146)]


In [33]:
plain_text_after_decryption = format_decrypted_plain_text(decypted_plain_text_blocks)
print("\nAfter decryption Plain Text :",plain_text_after_decryption)


After decryption Plain Text : Schrodinger's cat is dead and alive at the same time. This cat represents the (|psi_cat>=0.5*|psi_cat_alive> + 0.5*|psi_cat_dead>) i.e superposition of wave-functions representing cat being dead and alive at the same time.


### <a id='6.5'>5.5 Checking if decrypted plain_text is same as original plain_text</a>

In [34]:
if (original_plain_text == plain_text_after_decryption):
    print("\nHurrayyy!!!\n\nDecrypted plain_text is same as original plain_text! :) ")


Hurrayyy!!!

Decrypted plain_text is same as original plain_text! :) 


# <a id='final'>6. Final Remarks</a>
<a href='#index'>Go back to the top</a>

+ ASCII values :
    + ASCII values from 0-31 are not alpha-numeric characters.
    + ASCII values from ***32-99 contain [A_Z,a,b,c] and special characters such as @,#,',",space,etc***
    + ASCII values from ***100-126 contain [d-z] which are 3 digit.***
    + Since we're ***not using ASCII values from 0-31, we can convert ASCII values of [d-z] to 2-digit number, by subtracting 100 from them.***
    + If we subtract 100 from ASCII values of 'd', which is 100, we get 00. ***dam will be 009709 which 9709***, so 'd' is lost in the processing. 
    + Unused ASCII values we have now are [27,28,29,30,31], so we assign d=28. 
+ This program uses 25 digit prime number, so it can encrypt upto maximum of 12 characters at a time i.e 24 digits
+ So, to encrypt large plain text, ***RSA was implemented as a block cipher.***